<a href="https://colab.research.google.com/github/pchanial/python-for-data-scientists/blob/master/Course_Numpy_JAX_APC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Numpy & JAX

- Numpy: provides a powerful array model and vectorized functions for mathematical computations. It is CPU-only.

- JAX: JIT compilations of Python functions, and more (autodifferentiation, vectorization, parallelization). Can work on GPUs, TPUs.

Most of the Numpy library has been re-implemented in JAX, so if Numpy is familiar to you, JAX will be (with many caveats).

## Objectives of the course:
- Understand why Numpy is faster than Vanilla Python
- Understand why JAX is faster than Numpy
- Numpy concepts no more applicable to JAX
- Numpy concepts still applicable to JAX


In [9]:
# imports necessary for the course
import timeit
from typing import Any, Callable, Sequence
import jax
import jax.numpy as jnp
import jaxlib
import numpy as np
import matplotlib.pyplot as mp
#import polars as pl

print('Devices:', jax.devices())
import time


# helpers
def pointer(x: np.ndarray) -> int:
  """Returns the memory address of the first array element."""
  return x.__array_interface__['data'][0]


def bench_one(func: Callable[[], Any]) -> float:
    """Returns execution time in s."""
    repeat = 7
    timer = timeit.Timer(func)
    number, _ = timer.autorange()
    runs = np.array([_ / number for _ in timer.repeat(repeat=repeat, number=number)])
    runs_ms = runs * 1000
    print(f'{np.min(runs_ms):.3f} ms ± {np.std(runs_ms) * 1000:.2f} µs (min ± std. dev. of {repeat} runs, {number} loops each)')
    return np.min(runs)


def bench(func: Callable, values: Sequence[Any], *, setup: Callable | None = None) -> list[float]:
    elapsed_times = []
    for value in values:
        if setup is not None:
            args = setup(value)
        else:
            args = (value,)
        if isinstance(func, jaxlib.xla_extension.PjitFunction):
            func(*args)
            benchmarked_func = lambda: func(*args).block_until_ready()
        else:
            benchmarked_func = lambda: func(*args)
        elapsed_times.append(bench_one(benchmarked_func))
    return elapsed_times


def bench_many(funcs: Sequence[Callable], values: Sequence[Any], setups: Sequence[Callable | None], labels: Sequence[str]) -> None:
    for func, setup, label in zip(funcs, setups, labels):
        run_times = bench(func, values, setup=setup)
        mp.loglog(values, run_times, marker='.', label=label)
    mp.ylabel('Elapsed time [s]')
    mp.legend()


Devices: [CpuDevice(id=0)]


## Why is Numpy faster than Vanilla Python

![numpy-vs-python](https://raw.githubusercontent.com/pchanial/python-for-data-scientists/master/images/numpy-vs-python.png)

The Numpy array model is quite powerful, but before delving into the details on how Numpy arrays can be manipulated, it is interesting to understand why they are much more efficient than Python lists.

- first, when a Numpy array is created, its elements are stored one next to the other (the memory storage is contiguous, see figure on the left for a 2-dimensional array), whereas in a Python list, elements are created before the list and they can be stored wherever in the memory (the memory storage is scattered, see figure on the right). In most systems, data from the main memory is transferred to the CPU via layers of caches, which implies that memory transfers from the cache to the CPU involve whole chunks of contiguous memory (a cache line) even if only few bytes in the cache line are actually requested by the CPU. As a consequence, a non-contiguous memory storage of the data will force the transfer of unneeded data from the cache and will incur a bandwidth penalty.

- Vanilla Python memory layout

![list of list](https://raw.githubusercontent.com/pchanial/python-for-data-scientists/gh-pages/source/layout_listoflist.png)

- Numpy memory layout

![ndarray](https://raw.githubusercontent.com/pchanial/python-for-data-scientists/gh-pages/source/layout_2darray.png)

- modern architectures also have the possibility to anticipate transfers from the memory by prefetching the next cache lines. This mechanism will obviously better work when the data storage is contiguous.

- in Numpy, all elements occupy the same number of bytes, and as a consequence, the location of an element in the memory (its address) can be cheaply computed from its index and the location of the first element. There is no such relationship in Python lists: the location of each element has to be stored in the memory, so that every read or write access has the indirection overhead of transferring this element location to the CPU beforehand.

- SIMD instructions can operate on several data in a single CPU cycle, as long as they are contiguous

### Numpy array model

![Numpy array model](https://raw.githubusercontent.com/pchanial/python-for-data-scientists/master/images/numpy-array-model.png)



For writing efficient code for Numpy and JAX, one should think in terms of vectors, matrices or tensors to reduce the back and forth between the low-level implementation of mathematical functions and the Python interpreter.

### Example:
Numpy's C code uses a lot of macro and pre-processing to specialise the different datatypes and
contiguity configurations between the operands. This work is not done by JAX: outside of JIT all arrays are contiguous.

![Numpy loops](https://raw.githubusercontent.com/pchanial/python-for-data-scientists/master/images/numpy-loop.png)



**Exercice**: Wave the for loops goodbye!

Compute $\pi$, as given by the Madhava formula
$\pi = \sqrt{12}\sum^\infty_{k=0} \frac{(-\frac{1}{3})^{k}}{2k+1}$.
The $k$ indices ranging from 0 to (let’s say) 29 will be returned by the NumPy function `arange` (see above) and $\pi$ will be computed by calling another NumPy function (`sum`), instead of using a for loop.

In [ ]:
N = 30
pi = ...
assert abs(pi - np.pi) < 1e-15

## Why is JAX faster than Numpy

JAX has a Just-In-Time compiler (XLA). This compiler can perform many whole-program optimizations:
- operation fusion (or kernel fusion) is the flagship feature of XLA
- specializing to known tensor shapes, allowing for more aggressive constant propagation or loop unrolling
- analyzing and scheduling memory usage to eliminate intermediate storage buffers
- substituting inplace operations for copies
- removing dead branches and only computing subsets of requested values if not all of them are being returned

### Example 1: kernel fusion

With JAX, it is easy to inspect the computational graphs of compiled and non-compiled functions. The internal representation, which is fed to the XLA compiler is called HLO.

In this simple example, we have two sequential operations:
- multiply by 3
- add 2

```python
def f(x):
  return 3 * x + 2
```

- Non-compiled computation graph: the first operation is performed on the input array and the second one is applied on the result (like Numpy)

![HLO non-optimized](https://raw.githubusercontent.com/pchanial/python-for-data-scientists/master/images/hlo-non-optimized.png)

- Compiled computation graph: the for loops are fused, the tow operations are both performed at each iteration (unlike Numpy)

![HLO optimized](https://raw.githubusercontent.com/pchanial/python-for-data-scientists/master/images/hlo-optimized.png)



In [ ]:
values = [2**n for n in range(10, 24)]

setup_numpy = lambda n: (np.arange(n, dtype=np.float32),)
setup_jax = lambda n: (jnp.arange(n, dtype=np.float32),)
results = bench_many(
    [np.sum, jax.jit(jnp.sum)],
    values,
    setups=[setup_numpy, setup_jax],
    labels=['numpy', 'jax'],
)
mp.title('sum(x)')

In [ ]:
values = [2**n for n in range(10, 24)]

setup_numpy = lambda n: (np.random.normal(size=n).astype(np.float32), np.random.normal(size=n).astype(np.float32))
key = jax.random.key(0)
key1, key2 = jax.random.split(key)
setup_jax = lambda n: (jax.random.normal(key1, (n,), np.float32), jax.random.normal(key2, (n,), np.float32))

bench_many(
    [lambda x, y: 2 * x * y + 3 * y + 1, jax.jit(lambda x, y: 2 * x * y + 3 * y + 1)],
    values,
    setups=[setup_numpy, setup_jax],
    labels=['numpy', 'jax'],
)
mp.title('2xy + 3y + 1')


### Example 2: Jit can remove useless computations

In [ ]:
values = [4**n for n in range(5, 12)]

key = jax.random.key(0)
setup = lambda n: (jax.random.normal(key, (n,)),)


@jax.jit
def f(x):
    return jnp.exp(x)

@jax.jit
def f_subset(x):
    return jnp.exp(x)[:2]

bench_many(
    [f, f_subset],
    values,
    setups=[setup, setup],
    labels=['whole', 'subset'],
)



## Numpy concepts not applicable to JAX

1. Vues vs copies

Outside of jit (for non-compiled functions), JAX arrays are contiguous. For example, a slice results in a copy. Inside jit, the choice of having vues instead of copies is handed over to the compiler (XLA).

2. Inplace operations

JAX arrays are immutable, so they can only be copied outside of jit. Inside jit, it is again the responsability of the XLA compiler to perform the actual operations inplace.

3. Data type promotion rules

JAX and Numpy do not have the same promotion rules.
- Numpy favors the data integrity and will upcast in order not to loose information (versions < 2.0 even look at the values in some cases before upcasting)
- JAX is more pragmatic and broadly speaking tries to avoid changing the number of bytes encoding and have a predictable associative rules (unlike Numpy) for type promotion.

4. Non-number data types

Datetimes, string and object data types have no JAX equivalent. Use Pandas or Polars instead.

5. Structured arrays

Structured dtypes (aka records) are not implemented in JAX . Use JAX PyTrees.

4. Random library

The Numpy random library has not been ported to jax.numpy, because Numpy uses a global state for the pseudo random generator, which is incompatible with parallelization. Use jax.random instead.

## Numpy concepts still applicable to JAX

- indexing
- array creation
- array manipulation
- broadcasting
- ufunc methods



### Indexing

![Numpy indexing](https://raw.githubusercontent.com/pchanial/python-for-data-scientists/master/images/numpy-indexing.png)


### Array creation

![Array creation 1](https://raw.githubusercontent.com/pchanial/python-for-data-scientists/master/images/numpy-creation1.png)

![Array creation 2](https://raw.githubusercontent.com/pchanial/python-for-data-scientists/master/images/numpy-creation2.png)

![Array creation 3](https://raw.githubusercontent.com/pchanial/python-for-data-scientists/master/images/numpy-creation3.png)

![Array creation 4](https://raw.githubusercontent.com/pchanial/python-for-data-scientists/master/images/numpy-creation4.png)

![Array creation 5](https://raw.githubusercontent.com/pchanial/python-for-data-scientists/master/images/numpy-creation5.png)


Array manipulation

![Array manipulation](https://raw.githubusercontent.com/pchanial/python-for-data-scientists/master/images/numpy-array-manipulation.png)

![Array combining](https://raw.githubusercontent.com/pchanial/python-for-data-scientists/master/images/numpy-array-combining.png)



### Broadcasting
1. What is Broadcasting?

Broadcasting is a powerful feature of Numpy (and JAX) that allows operations (such as addition, multiplication etc.) which are normally element-wise to be carried out on arrays of different shapes without having to manually resize them. In other words, broadcasting automatically expands the dimensions of arrays to be the same size before operating them together. It can be seen as a generalization of operations involving an array and a scalar.

2. Advantages of Broadcasting

- Simplicity: Broadcasting simplifies the code by avoiding the need to manually resize tables.
- Performance: Broadcasting improves performance by avoiding data duplication.
- Readability: Broadcasting makes the code more concise

3. Examples

- the addition of a scalar on an matrix can be seen as the addition of a matrix with identical elements (and same dimensions).

![broadcast scalar](https://raw.githubusercontent.com/pchanial/python-for-data-scientists/gh-pages/source/broadcast_scalar.png)

- the addition of a row on a matrix will be seen as the addition of a matrix with replicated rows (the number of columns must match).

![broadcast column](https://raw.githubusercontent.com/pchanial/python-for-data-scientists/gh-pages/source/broadcast_column.png)

- conversely the addition of a column on a matrix will be seen as the addition of a matrix with replicated columns (the number of rows must match)

![broadcast row](https://raw.githubusercontent.com/pchanial/python-for-data-scientists/gh-pages/source/broadcast_row.png)

What if the rank of the arrays is greater than 2? There is no restriction on the rank: any dimension of length 1 is broadcastable and is virtually replicated to match the other array’s dimension length. The two arrays may have different broadcastable dimensions. If this happens, the result of the operation will have more elements than any of the operands.

Can it work on arrays of different ranks? Sure! Dimensions of length 1 are prepended (added on the left of the array shape) until the two arrays have the same rank. As a consequence, the following operation is possible:






In [ ]:
np.zeros((5, 9)) + np.ones(9)

but not this one, since the righmost dimensions are different:


In [ ]:
np.zeros((5, 9)) + np.ones(5)

To fix it, an additional dimension must be specified and added on the right:


In [ ]:
np.zeros((5, 9)) + np.ones(5)[:, None]  # or np.newaxis instead of None

Let's check that broadcasting does not require additional memory. There is no replication along the "missing" dimensions, because a stride of zero is used. This is why broadcasting is a very cheap operation.

In [5]:
a1 = np.ones((4, 1))
a2 = np.ones((2, 1, 3))
a1_broadcast, a2_broadcast = np.broadcast_arrays(a1, a2)
print('broadcast shapes:', a1_broadcast.shape, a2_broadcast.shape)
print('same memory layout:', pointer(a1) == pointer(a1_broadcast))
print(f'{a1.strides=}', f'{a1_broadcast.strides=}')

broadcast shapes: (2, 4, 3) (2, 4, 3)
same memory layout: True
a1.strides=(4, 4) a1_broadcast.strides=(0, 4, 0)


You should use broadcasting wherever possible, just keep an eye on the size of the broadcast result to make sure that it does not become too large.

In [ ]:
np.broadcast_shapes((3, 1, 4), (3, 4), (7, 1, 1, 1))

**Exercise 1**: Can the arrays of the following shapes be broadcast together? If yes, what would be the shape of the result?

    (7, 1) and (7, 4)
    (7,) and (4, 7)
    (3, 3) and (2, 3)**texte en gras**
    (1, 1, 1, 8) and (1, 9, 1)
    (4, 1, 9) and (3, 1)

**Exercise 2**: Remove the for loops in this code by using broadcasting and measure the improvement in execution time.


In [ ]:
import matplotlib.pyplot as mp
import numpy as np

NDETECTOR = 8
NSAMPLE = 1000
SAMPLING_PERIOD = 0.1
GLITCH_TAU = 0.3
GLITCH_AMPL = 20
GAIN_SIGMA = 0.03
SOURCE_AMPL = 7
SOURCE_PERIOD = 5
NOISE_SIGMA = 0.7

time = np.arange(NSAMPLE) * SAMPLING_PERIOD
glitch = np.zeros(NSAMPLE)
glitch[100:] = GLITCH_AMPL * np.exp(-time[:-100] / GLITCH_TAU)
gain = 1 + GAIN_SIGMA * np.random.standard_normal(NDETECTOR)
offset = np.arange(NDETECTOR)
source = SOURCE_AMPL * np.sin(2 * np.pi * time / SOURCE_PERIOD)
noise = NOISE_SIGMA * np.random.standard_normal((NDETECTOR, NSAMPLE))

signal = np.empty((NDETECTOR, NSAMPLE))
for idet in range(NDETECTOR):
    for isample in range(NSAMPLE):
        signal[idet, isample] = (
            gain[idet] * source[isample]
            + glitch[isample]
            + offset[idet]
            + noise[idet, isample]
        )

mp.figure()
mp.subplot(211)
mp.imshow(signal, aspect='auto', interpolation='none')
mp.xlabel('sample')
mp.ylabel('detector')
mp.subplot(212)
for s in signal:
    mp.plot(time, s)
mp.xlabel('time [s]')
mp.ylabel('signal')
mp.show()


**Exercise 3**: Write a one-liner function that normalizes by the euclidian norm M N-dimensional real vectors packed in an array of shape (M, N). Bonus if the function works with a tensor of any rank, such as (P, Q, M, N).

In [ ]:
import numpy as np

def fast_normalize(v):
    return v / ???

vectors = np.random.normal(size=(10, 3))
expected_normalized_vectors = vectors.copy()
for vector in expected_normalized_vectors:
    vector /= np.sqrt(vector[0]**2 + vector[1]**2 + vector[2]**2)
actual_normalized_vectors = fast_normalize(vectors)

assert np.allclose(actual_normalized_vectors, expected_normalized_vectors)

**Exercise 4**: Using broadcasting, write in one line the multiplication of a vector of size P with a tensor of arbitrary rank and of shape (P, Q, R, ...).

In [6]:
vector = np.array([1, 2, 3, 4])
tensor = np.arange(4*2*2*3).reshape(4, 2, 2, 3)

product = ???

assert np.allclose(product, np.array([vector[i] * tensor[i] for i in range(4)]))

### Universal function methods



![Ufunc 1](https://raw.githubusercontent.com/pchanial/python-for-data-scientists/master/images/numpy-ufunc1.png)

![Ufunc 2](https://raw.githubusercontent.com/pchanial/python-for-data-scientists/master/images/numpy-ufunc2.png)

